In [ ]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F

In [ ]:
import matplotlib
# matplotlib.use("TkAgg")
%matplotlib inline
import matplotlib.pyplot as plt

## 2D dataset

In [ ]:
num_points = 100
X1 = np.linspace(-2.5, 1.9, num_points)
X2 = np.linspace(-2.2, 2.1, num_points)
X1, X2 = np.meshgrid(X1, X2)
Y = np.sin(np.sqrt(X1**2 + X2**2))*2-1. - 0.1*(X1)+0.02*(X2)

####Scaling the data to range -1,1
X1 = 2*(X1 - X1.min())/(X1.max() - X1.min()) -1
X2 = 2*(X2 - X2.min())/(X2.max() - X2.min()) -1
Y = 2*(Y - Y.min())/(Y.max() - Y.min()) -1
Y = Y/2

x1 = X1.reshape(-1)
x2 = X2.reshape(-1)

xx = torch.Tensor(np.c_[x1, x2])
yy = torch.Tensor(Y.reshape(-1,1))


In [ ]:
# %matplotlib tk
fig = plt.figure()
ax = fig.add_subplot(projection = '3d')
ax.plot_surface(X1, X2, Y, cmap='plasma')
ax.set_xlabel('X1')
ax.set_ylabel('X2')
ax.set_zlabel('Y')
plt.show()

In [ ]:
import sys
sys.path.append("./Input-Invex-Neural-Network/")

In [ ]:
from classes import ConvexNN

In [ ]:
EPOCHS = 3000
actf = nn.ELU
learning_rate = 0.005
criterion = nn.MSELoss()

# Convex NN

In [ ]:
cvxNet = ConvexNN([2, 10, 10, 1], actf)
optimizer = torch.optim.Adam(cvxNet.parameters(), lr=learning_rate)

In [ ]:
%matplotlib inline
fig = plt.figure(figsize=(15,6))
ax = fig.add_subplot(121, projection='3d')
ax2 = fig.add_subplot(122)

for epoch in range(EPOCHS):

    yout = -cvxNet(xx)    
    loss = criterion(yout, yy)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    if epoch%100 == 0 or epoch==EPOCHS-1:
        print(f'Epoch: {epoch}, Loss:{float(loss)}')
        ax.clear()
        ax.scatter(X1, X2, yy.data.numpy().reshape(Y.shape), marker= '.')
        ax.scatter(X1, X2, yout.data.numpy().reshape(Y.shape), color='r', marker='.')
        ax2.clear()
        ax2.contourf(X1, X2, yout.data.numpy().reshape(Y.shape), levels=20)

        fig.canvas.draw()
        plt.pause(0.01)
plt.close()

In [ ]:
%matplotlib inline

y_ = yout.data.cpu().numpy().reshape(Y.shape)

fig = plt.figure(figsize=(8,6))
ax = fig.add_subplot(projection = '3d')
ax.view_init(49, -71)
ax.plot_surface(X1, X2, y_, cmap='plasma', alpha=0.9)
ax.set_xlabel('X1')
ax.set_ylabel('X2')
ax.set_zlabel('Y')
# plt.pause(0.1)
plt.show()

In [ ]:
cvxNet.eval()

## Plotting Contour Plot

In [ ]:
! mkdir outputs/09_convex_distance/

In [ ]:
# LVLs = np.linspace(sim.min(), sim.max(), 20)
LVLs = 50

y_ = cvxNet(xx).data.cpu().numpy().reshape(Y.shape)

plt.figure(figsize=(5,5))
plt.contourf(X1, X2, y_, levels=LVLs)
cs = plt.contour(X1, X2, y_, levels=LVLs, linestyles="None", colors="k", linewidths=1)
plt.clabel(cs, cs.levels[1::2], inline=True, fontsize=8, fmt="%1.2f")
minima = xx[y_.argmin()]
plt.scatter(*minima.tolist(), s=100, edgecolors="red")

# plt.tick_params(left = False, right = False , labelleft = False ,
#                 labelbottom = False, bottom = False)
# plt.savefig("./outputs/04.0_convex_distance/convex_contour_dist.pdf", bbox_inches='tight')

In [ ]:
min_val = y_.min()
min_val

In [ ]:
minima = xx[None, y_.argmin()]
minima

In [ ]:
# minima[0, 0] = minima[0, 0] + 0.25
# minima[0, 1] = minima[0, 1] - 0.1

## Finding Mimima by using gradient descent

In [ ]:
# xmin = torch.randn_like(minima)
xmin = minima.clone()
xmin = torch.autograd.Variable(xmin, requires_grad=True)
xmin

In [ ]:
optim = torch.optim.Adam([xmin], lr=0.001)
STEPS = 4000
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optim, T_max=STEPS)

### Training Loop

In [ ]:
for step in range(STEPS):
    optimizer.zero_grad() 

    ymin = cvxNet(xmin)
    xgrad = torch.autograd.grad(outputs=ymin, inputs=xmin, grad_outputs=torch.ones_like(ymin), 
                            only_inputs=True, retain_graph=True, create_graph=True)[0]
    xmin.grad = xgrad
    
    optim.step()
    scheduler.step()
    
    if step%100 == 0:
        print(f"STEPS: {step}, xmin: {xmin.data}, min_value: {ymin.data}")

In [ ]:
minima = xmin.data
minima_backup = minima
minval_backup = min_val

In [ ]:
# LVLs = np.linspace(sim.min(), sim.max(), 20)
LVLs = 50

y_ = cvxNet(xx).data.cpu().numpy().reshape(Y.shape) - min_val

plt.figure(figsize=(5,5))
plt.contourf(X1, X2, y_, levels=LVLs)
cs = plt.contour(X1, X2, y_, levels=LVLs, linestyles="None", colors="k", linewidths=1)
plt.clabel(cs, cs.levels, inline=True, fontsize=8, fmt="%1.2f")
plt.scatter(*minima[0].tolist(), s=100, edgecolors="red")
plt.scatter(*xmin.data[0].tolist(), s=50, edgecolors="red", marker="*")

# plt.tick_params(left = False, right = False , labelleft = False ,
#                 labelbottom = False, bottom = False)
# plt.savefig("./outputs/04.0_convex_distance/convex_contour_dist.pdf", bbox_inches='tight')

In [ ]:
# minima = xmin.data
# min_val = ymin.data[0].item()
# minima, min_val

## Using Learned function for Distance

In [ ]:
xx_ = xx + minima
y_ = cvxNet(xx_).data.cpu().numpy().reshape(Y.shape)- min_val
# y_ = y_**0.5

LVLs = 50
plt.figure(figsize=(5,5))
plt.contourf(X1, X2, y_, levels=LVLs)
cs = plt.contour(X1, X2, y_, levels=LVLs, linestyles="None", colors="k", linewidths=1)
plt.clabel(cs, cs.levels, inline=True, fontsize=10, fmt="%1.2f")
plt.scatter(0, 0, s=100, edgecolors="red")

## Contour as Distance

In [ ]:
def get_contour_distance_lp(level=1.0, p=1):
    xx_ =  xx
    x_norm = torch.norm(xx_, dim=1, keepdim=True)
    xx_= xx_/x_norm*level
    y_ = torch.norm(xx_, p=p, dim=1, keepdim=True)
    y_ = y_*x_norm#/level
    return y_

In [ ]:
y_ = get_contour_distance_lp(0.5, p=1.0).data.cpu().numpy().reshape(Y.shape)

LVLs = 50
plt.figure(figsize=(8,8))
plt.contourf(X1, X2, y_, levels=LVLs)
cs = plt.contour(X1, X2, y_, levels=LVLs, linestyles="None", colors="k", linewidths=1)
plt.clabel(cs, cs.levels, inline=True, fontsize=10, fmt="%1.2f")
# plt.scatter(0, 0, s=100, edgecolors="red")

### Extracting the Exact Contour 

In [ ]:
contour_levels = []

In [ ]:
cvxNet.eval()

In [ ]:
xx.shape

In [ ]:
level = 0.05 #0.05   ### add more contours here
xx_norm = torch.norm(xx, dim=1, keepdim=True)
xx_ = xx/xx_norm * level

In [ ]:
xx_lnorm = torch.ones_like(xx_norm)*2#0.5
xx_lnorm = torch.autograd.Variable(xx_lnorm, requires_grad=True)
xx_lnorm

In [ ]:
# # y_ = (xx_norm/xx_lnorm.data).cpu().reshape(X1.shape)
# yy_ = cvxNet(xx_*xx_lnorm + minima) - min_val - level
# # yy_ = torch.abs(yy_)
# y_= yy_.data.numpy().reshape(Y.shape)

# LVLs = 20
# plt.figure(figsize=(6,6))
# plt.contourf(X1, X2, y_, levels=LVLs)
# cs = plt.contour(X1, X2, y_, levels=LVLs, linestyles="None", colors="k", linewidths=1)
# plt.clabel(cs, cs.levels, inline=True, fontsize=10, fmt="%1.2f")

In [ ]:
optimizer = torch.optim.Adam([xx_lnorm], lr=0.03)
STEPS = 4000
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=STEPS)

In [ ]:
# minima -= 0.01

### Training Loop

In [ ]:
for step in range(STEPS):
    optimizer.zero_grad() 

    yy_ = torch.abs(cvxNet(xx_*xx_lnorm + minima) - min_val - level)
    xgrad = torch.autograd.grad(outputs=yy_, inputs=xx_lnorm, grad_outputs=torch.ones_like(yy_), 
                            only_inputs=True, retain_graph=True, create_graph=True)[0]
    xx_lnorm.grad = xgrad
    
    optimizer.step()
    scheduler.step()    
    
    if step%100 == 0:
        print(f"STEPS: {step}, norm_min: {xx_lnorm.data.min()}, norm_max: {xx_lnorm.data.max()}, min_value: {ymin.data.min()}")

In [ ]:
y_ = (xx_norm/xx_lnorm.data).cpu().reshape(X1.shape)

LVLs = 20
plt.figure(figsize=(6,6))
plt.contourf(X1, X2, y_, levels=LVLs)
cs = plt.contour(X1, X2, y_, levels=LVLs, linestyles="None", colors="k", linewidths=1)
plt.clabel(cs, cs.levels, inline=True, fontsize=10, fmt="%1.2f")

# plt.tick_params(left = False, right = False , labelleft = False ,
#                 labelbottom = False, bottom = False)
plt.savefig(f"./outputs/09_convex_distance/convex_contour_dist_l{level}.pdf", bbox_inches='tight')

In [ ]:
y_.shape

In [ ]:
contour_levels += [level]
contour_levels = list(set(contour_levels))

In [ ]:
contour_levels

In [ ]:
contour_levels = sorted(contour_levels)

## Convex Function for refrence

In [ ]:
# y_ = cvxNet(xx + minima).data.cpu().numpy().reshape(Y.shape)- min_val
y_ = cvxNet(xx).data.cpu().numpy().reshape(Y.shape) - min_val

LVLs = 50
plt.figure(figsize=(6,6))
plt.contourf(X1, X2, y_, levels=LVLs)
cs = plt.contour(X1, X2, y_, levels=LVLs, linestyles="None", colors="k", linewidths=1)
plt.clabel(cs, cs.levels[:10], inline=True, fontsize=10, fmt="%1.2f")
plt.contour(X1, X2, y_, levels=contour_levels, linestyles="None", colors="orange", linewidths=2)
plt.scatter(*minima[0].tolist(), s=100, edgecolors="red")

# plt.tick_params(left = False, right = False , labelleft = False ,
#                 labelbottom = False, bottom = False)
plt.savefig(f"./outputs/09_convex_distance/convex_contour_dist_func.pdf", bbox_inches='tight')

In [ ]:
minima